### Importing Libraries

In [66]:
# smart LLM imports
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_experimental.smart_llm import SmartLLMChain
# tree of thoughts imports
from typing import Tuple
from langchain_experimental.tot.base import ToTChain
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity

import load_dotenv
load_dotenv.load_dotenv()

True

### A tricky question to start with

In [56]:
hard_question = """
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
"""

### Normal prompting technique

In [49]:
np = PromptTemplate.from_template("Answer user's question. {question}")

### answer from GPT 3.5 turbo

In [57]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-0613', temperature = '1')
llm_chain = LLMChain(prompt=np, llm=llm)
print(llm_chain(hard_question)['text'])

The ball is in the cup, which Bob left in the garden.


### answer from Vertex AI

In [53]:
llm = VertexAI(model_name = 'text-bison-32k', temperature = '0.2')
llm_chain = LLMChain(prompt=np, llm=llm)
print(llm_chain(hard_question)['text'])

 The ball is in the garden.


### answer from GPT 4

In [58]:
llm = ChatOpenAI(model_name = 'gpt-4', temperature = '1')
llm_chain = LLMChain(prompt=np, llm=llm)
print(llm_chain(hard_question)['text'])

The ball is in the bedroom.


### Ideation -> Critique -> Resolve technique

In [59]:
prompt_template = 'Don''t jump to the conclusion immediately, analyze step by step, apply different reasoning strategy: """{question}?"""'
prompt_template = PromptTemplate.from_template(prompt_template)
llm = VertexAI(model_name = 'text-bison', temperature = '0.7')
smart_chain = SmartLLMChain(ideation_llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-0613', temperature = '1'),
                    critique_llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-0613', temperature = '0.7'),
                    resolver_llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-0613', temperature = '0'),
                    prompt=prompt_template, n_ideas=3, verbose=True)
smart_chain.run(hard_question)



> Entering new SmartLLMChain chain...
Prompt after formatting:
Dont jump to the conclusion immediately, analyze step by step, apply different reasoning strategy: """
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
?"""
Idea 1:
1. Bob starts in the living room.
2. He walks to the kitchen, carrying a cup.
3. He puts a ball in the cup.
4. He carries the cup with the ball to the bedroom.
5. He turns the cup upside down.
6. He walks to the garden.
7. He puts the cup down in the garden.
8. He walks to the garage.

Based on the given information, we can conclude that the ball is still in the bedroom. The ball was placed in the cup in the kitchen, but Bob never took the ball out of the cup before putting it down in the garden. Therefore, the ball must still be in the cup in th

'Idea 3: 1. Bob is in the living room.\n2. He walks to the kitchen, carrying a cup.\n3. He puts a ball in the cup and carries the cup to the bedroom.\n4. He turns the cup upside down, then walks to the garden.\n5. He puts the cup down in the garden, then walks to the garage.\n\nBased on the given information, we can conclude that the ball is in the bedroom. This is because Bob puts the ball in the cup in the kitchen, carries the cup to the bedroom, and then turns the cup upside down in the garden. Therefore, the ball must still be in the place where he last had the cup, which is the bedroom.\n\nImproved Answer: The ball is in the bedroom, as Bob puts the ball in the cup in the kitchen, carries the cup to the bedroom, and then turns the cup upside down in the garden. Therefore, the ball must still be in the bedroom.\n\nFull Answer: The ball is in the bedroom. Bob puts the ball in the cup in the kitchen, carries the cup to the bedroom, and then turns the cup upside down in the garden. Th

### Tree of Thought technique

In [60]:
class check_answer(ToTChecker):
    def evaluate(self, problem_description: str, thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        last_thought = thoughts[-1]
        if 'bedroom' in last_thought:
            return ThoughtValidity.VALID_FINAL
        else:
            return ThoughtValidity.INVALID
            
llm = VertexAI(model_name = 'text-bison', temperature = '0.7')
tot_chain = ToTChain(llm=llm, checker=check_answer(), k=10, c=3, verbose=True, verbose_llm=True)
tot_answer = tot_chain.run(problem_description=hard_question)



> Entering new ToTChain chain...
Starting the ToT solve procedure.


> Entering new ProposePromptStrategy chain...
Prompt after formatting:
You are an intelligent agent that is generating thoughts in a tree of
thoughts setting.

The output should be a markdown code snippet formatted as a JSON list of
strings, including the leading and trailing "```json" and "```":

```json
[
    "<thought-1>",
    "<thought-2>",
    "<thought-3>"
]
```

PROBLEM


Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?




Possible next 3 valid thoughts based on the PROBLEM:


/Users/_blitzzz/Documents/GitHub/llm-meetup-demo/llm_venv/lib/python3.9/site-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Thought: The ball is in the cup.
Thought: The ball is in the bedroom.

> Finished chain.
